In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from collections import defaultdict
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
)
from modules.stochastic_program.factory import StochasticProgramFactory



### Prepare scenario data
Make scenarios complete cross product

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
scenarios

demand
scenarios start_hex_ids   end_hex_ids     time     vehicle_types        
0         861fa1817ffffff 861fa1817ffffff 00:00:00 bicycle             0
                                                   car                 0
                                                   kick_scooter        0
                                          04:00:00 bicycle             0
                                                   car                 0
...                                                                  ...
7         861fa1d6fffffff 861fa1d6fffffff 16:00:00 car                 0
                                                   kick_scooter        0
                                          20:00:00 bicycle             0
                                                   car                 0
                                                   kick_scooter        1

[51984 rows x 1 columns]

In [5]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [6]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [7]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [8]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [9]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [10]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Get Model Parameters


In [11]:
demand = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

In [12]:
start = timer()

for _, row in scenarios.reset_index().iterrows():
    demand[row.start_hex_ids][row.end_hex_ids][row.time.hour][row.vehicle_types][row.scenarios] = row.demand

end = timer()
print(f"Successfully converted dataframe into dictionary in {(end - start):.2f} seconds")

Successfully converted dataframe into dictionary in 3.72 seconds


In [13]:
distances = pd.read_pickle(PATH_DISTANCES)

In [14]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [15]:
start = timer()

vehicle_types = list(scenarios.reset_index()['vehicle_types'].unique())

for _, row in distances.reset_index().iterrows():
    for vehicle_type in vehicle_types:
        cost[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"cost_{vehicle_type}"]
        profit[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"profit_{vehicle_type}"]

end = timer()

In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [18]:
vehicle_types = ['kick_scooter']
FLEET_CAPACITY = {
    "kick_scooter": 200,
    # "bicycle": 100,
    # "car": 50,
}
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df, vehicle_types)
factory.set_initial_allocation(FLEET_CAPACITY)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.03 seconds
_convert_demand finished in 3.67 seconds
_convert_nodes finished in 0.01 seconds
_convert_parameters finished in 3.71 seconds
_set_max_demand finished in 0.21 seconds
set_initial_allocation finished in 0.00 seconds


In [19]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [20]:
stochastic_program.relocations_disabled = False

In [21]:
stochastic_program.create_model()

create_variables finished in 0.16 seconds
create_objective_function finished in 0.35 seconds
create_demand_constraints finished in 0.19 seconds
create_relocation_binary_constraints finished in 0.01 seconds
create_big_u_sum_constraints finished in 0.04 seconds
create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
create_no_refused_demand_constraints finished in 0.01 seconds
create_max_trips_constraints finished in 0.11 seconds


KeyError: 24

[0, 4, 8, 12, 16, 20, 32]

In [ ]:
stochastic_program.solve(logPath='./log')

/home/moritz/miniconda3/envs/VR/lib/python3.8/site-packages/pulp/apis/coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')


Status: Optimal
Optimal Value of Objective Function:  8020.439798004068
Runtime without preprocessing: 0.81 seconds
solve finished in 1.45 seconds


# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [ ]:
capacities = [
    {
        "kick_scooter": 200,
        "bicycle": 100,
        "car": 50,
    },
    # {
    #     "kick_scooter": 1000,
    #     "bicycle": 500,
    #     "car": 250,
    # },{
    #     "kick_scooter": 2000,
    #     "bicycle": 1000,
    #     "car": 500,
    # }
]


In [ ]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.03 seconds
_convert_demand finished in 2.96 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 2.99 seconds
_set_max_demand finished in 0.16 seconds
Status: Optimal
Optimal Value of Objective Function:  11870.79985320831
Runtime without preprocessing: 2.85 seconds
solve finished in 6.53 seconds


Status: Optimal
Optimal Value of Objective Function:  11912.732143256162
Runtime without preprocessing: 2.29 seconds
solve finished in 6.59 seconds


Status: Optimal
Optimal Value of Objective Function:  2308.0501597276566
Runtime without preprocessing: 1.97 seconds
solve finished in 3.17 seconds


Status: Optimal
Optimal Value of Objective Function:  2335.6483549727113
Runtime without preprocessing: 1.63 seconds
solve finished in 2.86 seconds




In [ ]:
results_df = pd.DataFrame.from_dict(results)

In [ ]:
results_df

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,11870.799853,2.851452,12223,7766,1722,2855,200,100,50,False,False
1,Optimal,11912.732143,2.294998,12240,7745,1702,2858,200,100,50,False,True
2,Optimal,2308.050160,1.966662,3866,28740,12934,0,200,100,50,True,False
3,Optimal,2335.648355,1.626799,3877,28710,12923,0,200,100,50,True,True


In [ ]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [ ]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand = last_demand_unstacked.stack('vehicle_types')


In [ ]:
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    current_fleet_capacity = {
        current_vehicle_type: FLEET_CAPACITY[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(lambda x: current_vehicle_type),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


Status: Optimal
Optimal Value of Objective Function:  3059.362886891975
Runtime without preprocessing: 0.87 seconds
solve finished in 1.58 seconds
Status: Optimal
Optimal Value of Objective Function:  738.8616900561902
Runtime without preprocessing: 0.86 seconds
solve finished in 1.52 seconds
Status: Optimal
Optimal Value of Objective Function:  2675.4223257152826
Runtime without preprocessing: 0.87 seconds
solve finished in 1.37 seconds


In [ ]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(FLEET_CAPACITY)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  11870.79985320831
Runtime without preprocessing: 2.81 seconds
solve finished in 6.48 seconds


In [ ]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,3059.362887,0.867127,2051,8440,36,313,['car']
1,Optimal,738.861690,0.863901,4034,4406,92,674,['bicycle']
2,Optimal,2675.422326,0.869983,3678,728,0,5922,['kick_scooter']
3,Optimal,11870.799853,2.808456,12223,7766,1722,2855,"[kick_scooter, bicycle, car]"


In [ ]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,11870.799853,2.808456,12223,7766,1722,2855,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,6473.646903,2.601011,9763,13574,128,6909,['car']['bicycle']['kick_scooter']


## Value Of The Stochastic Solution

In [ ]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [ ]:
results = []
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

In [ ]:
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  11558.41160926391
Runtime without preprocessing: 0.30 seconds
solve finished in 0.76 seconds
Status: Optimal
Optimal Value of Objective Function:  13194.734997859752
Runtime without preprocessing: 0.26 seconds
solve finished in 0.72 seconds


In [ ]:
results = pd.DataFrame.from_dict(results)

In [ ]:
results

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,11558.411609,0.303382,1503,873,256,341,"[kick_scooter, bicycle, car]"
1,Optimal,13194.734998,0.259457,1572,1182,165,363,"[kick_scooter, bicycle, car]"


In [ ]:
scenarios.reset_index().nunique()

scenarios         8
start_hex_ids    17
end_hex_ids      17
time              6
vehicle_types     3
demand           51
dtype: int64